# Проект speech-to-text

Я возьму датасет с русскоязычными аудиозаписями по ссылке https://disk.yandex.ru/d/v2Hipv7XG4fEDQ, применю к нему предобученную модель whisper-small из Hugging Face для распознавания речи и выведу транскрипции для 10 случайно выбранных аудиофайлов.


In [1]:
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    T5ForConditionalGeneration,
    T5Tokenizer
)
import sentencepiece
import torch
import torchaudio
import librosa
import os
import random
from datasets import load_dataset
import requests
from pathlib import Path
from tqdm import tqdm
import io
import soundfile as sf

c:\Users\ksbal\Desktop\HW\GenDL\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Устанавливаем устройство
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Загрузка модели whisper
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
forced_decoder_ids = whisper_processor.get_decoder_prompt_ids(language="ru", task="transcribe")
whisper_model.to(device).eval()

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [51]:
tsv_path = r"urls_normalized.tsv"
# Чтение ссылок
with open(tsv_path, "r", encoding="utf-8") as f:
    lines = [line.strip().split("\t")[0] for line in f if line.strip()]

In [52]:
len(lines)

100

In [5]:
lines[:10]

['http://storage.mds.yandex.net:80/get-voicetoloka/1872575/197f271b-b23f-4ee0-b240-e956a172d7af',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1879367/3d8c8d43-f7f2-479b-a857-c90faa5e2faf',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/45161c4c-3f2c-4638-940e-a69404074ebb',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/3c0ebd62-2cc1-4c9c-be63-5733511a11cd',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/6385560c-7068-45c2-9ba7-9e061249e0a4',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/c4f3178c-14b2-44f8-b36e-f34a47520e10',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/0f2ef6c9-3dec-45ae-b456-8115c9419044',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/4711cdac-8181-4f3c-8889-77cf408f3ff2',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/37e917e1-1fdf-4064-a74f-57a84bcb28b9',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/d203c652-509b-4c41-bdaf-f374e1c3c87e']

In [6]:
"""
Теперь выберем 10 случайных датасетов, чтобы вывести результат работы модели
"""
random.seed(42)
selected_urls = random.sample(lines, k=min(10, len(lines)))

In [7]:
def transcribe_audio(url: str) -> str:
    """Распознаёт речь из аудио по URL с помощью Whisper"""
    response = requests.get(url, timeout=15)
    response.raise_for_status()

    audio_bytes = io.BytesIO(response.content)
    audio_np, sr = sf.read(audio_bytes, dtype='float32')

    if audio_np.ndim > 1:
        audio_np = audio_np.mean(axis=1)

    if sr != 16000:
        audio_np = librosa.resample(audio_np, orig_sr=sr, target_sr=16000)

    input_features = whisper_processor(
        audio_np, sampling_rate=16000, return_tensors="pt"
    ).input_features.to(device)

    with torch.no_grad():
        predicted_ids = whisper_model.generate(
            input_features,
            forced_decoder_ids=forced_decoder_ids,
            max_length=448
        )

    transcription = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription.strip()

In [9]:
for i, url in enumerate(selected_urls, 1):
    try:
        print(f"\n[{i}] URL: {url}")
        raw_text = transcribe_audio(url)
        print(f"Распознано:     {raw_text}")

    except Exception as e:
        print(f"Ошибка: {e}")


[1] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/eebe7151-43f7-4e5f-af3d-d7a1a3ab2197
Распознано:     Александр Владимирович Попов

[2] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/fcceeeb4-5bb7-460a-853c-99c3c7bd5aef
Распознано:     Андрей Сахаров

[3] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/3c0ebd62-2cc1-4c9c-be63-5733511a11cd
Распознано:     Дайон Уорвик

[4] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/09c0c412-2316-4bf4-9f92-8349067de618
Распознано:     Брюс Springsteam

[5] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/47b9b402-e832-438d-ad85-3f7375867e4a
Распознано:     КОУ ПОРТЕР

[6] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/86aaebac-a43a-4f80-a6e4-136bfb5492e3
Распознано:     Голова ломка.

[7] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/373c13d2-0039-4ca1-a549-1563d2a8ef0a
Распознано:     Крестный отец 2

[8] URL: http://storage.mds.yandex.net:80/ge

Если послушать аудио и посмотрет на текст, то можно заметить, что есть некоторые ошибки

In [10]:
# Загрузка модели исправления ошибок
spell_tokenizer = T5Tokenizer.from_pretrained("UrukHan/t5-russian-spell")
spell_model = T5ForConditionalGeneration.from_pretrained("UrukHan/t5-russian-spell")
spell_model.to(device).eval()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [11]:
def correct_spelling(text: str) -> str:
    """Исправляет орфографические ошибки в тексте с помощью T5"""
    if not text:
        return ""
    
    input_for_model = "Spell correct: " + text
    inputs = spell_tokenizer(
        input_for_model,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    ).to(device)

    with torch.no_grad():
        outputs = spell_model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )

    corrected = spell_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected.strip()

In [12]:
for i, url in enumerate(selected_urls, 1):
    try:
        print(f"\n[{i}] URL: {url}")
        raw_text = transcribe_audio(url)
        print(f"Распознано:     {raw_text}")

        corrected_text = correct_spelling(raw_text)
        print(f"Исправлено:     {corrected_text}")

    except Exception as e:
        print(f"Ошибка: {e}")


[1] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/eebe7151-43f7-4e5f-af3d-d7a1a3ab2197
Распознано:     Александр Владимирович Попов
Исправлено:     Александр Владимирович Попов

[2] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/fcceeeb4-5bb7-460a-853c-99c3c7bd5aef
Распознано:     Андрей Сахаров
Исправлено:     Андрей Сахаров

[3] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/3c0ebd62-2cc1-4c9c-be63-5733511a11cd
Распознано:     Дайон Уорвик
Исправлено:     Дайон Уоррен

[4] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/09c0c412-2316-4bf4-9f92-8349067de618
Распознано:     Брюс Springsteam
Исправлено:     Брюс. ringsteam.

[5] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/47b9b402-e832-438d-ad85-3f7375867e4a
Распознано:     КОУ ПОРТЕР
Исправлено:     КОУ ПОРТЕР

[6] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/86aaebac-a43a-4f80-a6e4-136bfb5492e3
Распознано:     Голова ломка.
Исправлено:     

Я соберу датасет для дообучения, используя бесплатный API Groq (или альтернативную локальную LLM, например, через Ollama, если Groq недоступен).
Создам датасет из >1000 примеров исправления опечаток, сохраню его локально, загружу в Hugging Face через библиотеки datasets и huggingface_hub, и использую для дообучения модели с целью улучшить её качество по сравнению с предобученной версией.

In [13]:
import os
import json
import time
from groq import Groq, RateLimitError
from datasets import load_dataset, Dataset

In [ ]:
os.environ["GROQ_API_KEY"] = "****"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [15]:
PROMPT = (
    'Сгенерируй ровно одну строку в формате JSON:\n'
    '{"correct": "грамотное предложение на русском языке", "error": "то же предложение, но с 1–3 реалистичными опечатками '
    '(например, пропущенные буквы, переставленные соседние буквы, замена \'е\' на \'ё\' или \'и\' на \'й\', опечатки как при быстрой печати)"}\n'
    'Предложение должно быть простым, разговорным, длиной 3–10 слов. Не используй кавычки внутри значений. Не добавляй пояснений. Используй меньше местоимений "я" и "мне"'
)


In [16]:
def generate_pair():
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
            messages=[{"role": "user", "content": PROMPT}],
            model="llama-3.3-70b-versatile",
            temperature=0.8,
            max_tokens=120
        )

            print(response.choices[0].message.content)
            content = response.choices[0].message.content.strip()
            
            if content.startswith("```"):
                content = content.split("\n", 1)[1].rsplit("\n", 1)[0]
            data = json.loads(content)
            if "correct" in data and "error" in data:
                return {"error": data["error"], "correct": data["correct"]}
            
        except RateLimitError:
            wait_time = 5 ** attempt  
            print(f"Лимит исчерпан. Ждём {wait_time} сек...")
            time.sleep(wait_time)
        except Exception as e:
            print(f"Ошибка при генерации: {e}")
            time.sleep(1)
    return None

In [ ]:
# dataset = []
# target_size = 1100

# for i in range(target_size):
#     print(f"Генерация {i+1}/{target_size}")
#     pair = generate_pair()
#     if pair:
#         dataset.append(pair)
#         # Сохраняем сразу после каждого успешного примера
#         with open("spell_dataset.jsonl", "a", encoding="utf-8") as f:
#             f.write(json.dumps(pair, ensure_ascii=False) + "\n")

In [17]:
# Посмотрим на кол-во строк в получившемся файле
dataset = load_dataset("json", data_files="spell_dataset.jsonl")

print(len(dataset["train"]))

1147


In [ ]:
hf_dataset = Dataset.from_list(dataset["train"])

# Сохранение на Hugging Face Hub
# Сначала залогиньтесь:
# huggingface-cli login

# HF_REPO_ID = "ksbal/spell_dataset"

# try:
#     hf_dataset.push_to_hub(
#         repo_id=HF_REPO_ID,
#         token="***", 
#         private=False,
#         commit_message="Add correction dataset"
#     )
#     print(f"Датасет загружен: https://huggingface.co/datasets/{HF_REPO_ID}")
# except Exception as e:
#     print(f"Ошибка при загрузке на HF: {e}")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 894.12ba/s]
Processing Files (1 / 1): 100%|██████████| 26.4kB / 26.4kB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.36s/ shards]
c:\Users\ksbal\Desktop\HW\GenDL\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ksbal\.cache\huggingface\hub\datasets--ksbal--spell_dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admin

Датасет загружен: https://huggingface.co/datasets/ksbal/spell_dataset


Я дообучу выбранную модель (например, ruGPT3-small или другую) на собранном датасете с исправлениями опечаток. Затем протестирую её на аудиозаписях, в которых whisper-small допустил ошибки.

Для оценки качества я выведу 10 пар: исходный текст от Whisper и его исправленную версию, полученную с помощью дообученной модели.

In [19]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [20]:
raw_dataset = load_dataset("json", data_files="spell_dataset.jsonl")
len(raw_dataset["train"])

1147

In [21]:
# Модель
model_name = "UrukHan/t5-russian-spell"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [22]:
# Токенизация
def preprocess_function(examples):
    inputs = ["Spell correct: " + text for text in examples["error"]]
    targets = examples["correct"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = raw_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["error", "correct"]
)

In [23]:
# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [24]:
# Аргументы обучения
training_args = Seq2SeqTrainingArguments(
    output_dir="./spell_correction_finetuned",
    save_strategy="steps",
    save_steps=500,
    per_device_train_batch_size=8,       
    gradient_accumulation_steps=2,
    learning_rate=3e-5,
    num_train_epochs=3,
    logging_steps=100,
    fp16=torch.cuda.is_available(),      
    predict_with_generate=True,
    push_to_hub=False,
    report_to="none",
    remove_unused_columns=False,
)

In [25]:
# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\ksbal\AppData\Local\Temp\ipykernel_8096\1345063306.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [26]:
# Запускаем обучение
trainer.train()

# Сохраняем дообученную модель
trainer.save_model("./spell_correction_finetuned")
tokenizer.save_pretrained("./spell_correction_finetuned")

Step,Training Loss
100,1.078700
200,0.006300


c:\Users\ksbal\Desktop\HW\GenDL\venv\Lib\site-packages\transformers\modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./spell_correction_finetuned\\tokenizer_config.json',
 './spell_correction_finetuned\\special_tokens_map.json',
 './spell_correction_finetuned\\spiece.model',
 './spell_correction_finetuned\\added_tokens.json')

In [27]:
# Загружаем дообученную модель
finetuned_model_path = "./spell_correction_finetuned"
ft_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)
ft_model = AutoModelForSeq2SeqLM.from_pretrained(finetuned_model_path).to(device)

def correct_with_finetuned(text: str) -> str:
    if not text:
        return ""
    input_text = "Spell correct: " + text
    inputs = ft_tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)
    with torch.no_grad():
        outputs = ft_model.generate(**inputs, max_length=128, num_beams=4)
    return ft_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# Получаем 10 исходных текстов
transcriptions = []
for url in selected_urls:
    try:
        transcriptions.append(transcribe_audio(url))
    except:
        transcriptions.append("")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [28]:
# Выводим сравнение
for i, raw in enumerate(transcriptions, 1):
    orig_corr = correct_spelling(raw) 
    ft_corr = correct_with_finetuned(raw)  
    
    print(f"\n[{i}] Распознано:     {raw}")
    print(f"    Предобученная:  {orig_corr}")
    print(f"    Дообученная:    {ft_corr}")


[1] Распознано:     Александр Владимирович Попов
    Предобученная:  Александр Владимирович Попов
    Дообученная:    Александр Владимирович Попов

[2] Распознано:     Андрей Сахаров
    Предобученная:  Андрей Сахаров
    Дообученная:    Андрей Сахаров

[3] Распознано:     Дайон Уорвик
    Предобученная:  Дайон Уоррен
    Дообученная:    Дайон Уорвик

[4] Распознано:     Брюс Springsteam
    Предобученная:  Брюс. ringsteam.
    Дообученная:    Брюс

[5] Распознано:     КОУ ПОРТЕР
    Предобученная:  КОУ ПОРТЕР
    Дообученная:    КОМУ ПОРТЕР

[6] Распознано:     Голова ломка.
    Предобученная:  Голова ломка.
    Дообученная:    Голова ломка

[7] Распознано:     Крестный отец 2
    Предобученная:  Крестный отец 2
    Дообученная:    Крестный отец 2

[8] Распознано:     Дай не трехо.
    Предобученная:  Дай не трех.
    Дообученная:    Дай не трех

[9] Распознано:     Эффект бабочки.
    Предобученная:  Эффект бабочки.
    Дообученная:    Эффект бабочки

[10] Распознано:     Старший сы

Я посчитаю метрики и при необходимости вернусь к дообучению модели. На этом шаге оценивается корректность выполнения, а не сами значения метрик.

а) Используя эталонные транскрипции из этого датасета, я вычислю WER только для тех аудиофайлов, для которых предоставлены правильные ответы, — сначала для исходной модели whisper-small.

б) Затем я оценю WER для конвейера: whisper-small + предобученная модель исправления опечаток (выбранная мной самостоятельно).

в) Наконец, я посчитаю WER для конвейера: whisper-small + моя дообученная модель (на данных, собранных и дообученных мной самостоятельно).

In [34]:
with open("result_array.json", "r", encoding="utf-8") as f:
    raw_ground_truth = json.load(f)

In [35]:
len(raw_ground_truth)

96

In [36]:
raw_ground_truth[0]

{'url': 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/197f271b-b23f-4ee0-b240-e956a172d7af',
 'text': 'жизнь других'}

In [37]:
# Преобразуем список в словарь: {url: text}
ground_truth = {item["url"]: item["text"] for item in raw_ground_truth}

In [42]:
ground_truth

{'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/197f271b-b23f-4ee0-b240-e956a172d7af': 'жизнь других',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1879367/3d8c8d43-f7f2-479b-a857-c90faa5e2faf': 'элтон джон',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/45161c4c-3f2c-4638-940e-a69404074ebb': 'побег из шоушенка',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/6385560c-7068-45c2-9ba7-9e061249e0a4': 'мухаммед али',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/c4f3178c-14b2-44f8-b36e-f34a47520e10': 'дневник памяти',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/0f2ef6c9-3dec-45ae-b456-8115c9419044': 'рэйф файнс',
 'http://storage.mds.yandex.net:80/get-voicetoloka/1872575/4711cdac-8181-4f3c-8889-77cf408f3ff2': 'нефть',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/37e917e1-1fdf-4064-a74f-57a84bcb28b9': 'золотая лихорадка',
 'http://storage.mds.yandex.net:80/get-voicetoloka/2021744/d203c652-509b-4c41-bdaf-f374

In [45]:
# Фильтр для правильных ответов
filtered_lines = [url for url in lines if url in ground_truth]

print(f"Отфильтровано {len(filtered_lines)} URL с эталонными транскрипциями")

Отфильтровано 96 URL с эталонными транскрипциями


In [46]:
from torcheval.metrics import WordErrorRate

In [47]:
def compute_wer(predictions, references):
    metric = WordErrorRate()
    metric.update(predictions, references)
    return metric.compute().item()

In [48]:
# Собираем эталоны и предсказания
refs = []
preds_whisper = []
preds_pretrained = []
preds_finetuned = []

for url in filtered_lines:
    ref = ground_truth[url].strip().lower()
    try:
        raw = transcribe_audio(url).strip().lower()
        corr_pre = correct_spelling(raw).strip().lower()
        corr_ft = correct_with_finetuned(raw).strip().lower()
        
        refs.append(ref)
        preds_whisper.append(raw)
        preds_pretrained.append(corr_pre)
        preds_finetuned.append(corr_ft)
    except Exception as e:
        print(f"Ошибка {e}")
        

In [50]:
wer_whisper = compute_wer(preds_whisper, refs)
wer_pretrained = compute_wer(preds_pretrained, refs)
wer_finetuned = compute_wer(preds_finetuned, refs)

print(f"WER (Whisper-small):          {wer_whisper:.4f}")
print(f"WER (Whisper + предобученная): {wer_pretrained:.4f}")
print(f"WER (Whisper + дообученная):   {wer_finetuned:.4f}")

WER (Whisper-small):          0.4957
WER (Whisper + предобученная): 0.8087
WER (Whisper + дообученная):   0.4217


Я проведу максимально глубокий research и серию экспериментов в рамках бесплатного Google Colab, чтобы добиться наилучшего качества решения задачи speech-to-text:

* Подберу и протестирую альтернативные предобученные ASR-модели, сравнивая их WER на эталонных транскрипциях, чтобы найти ту, что даёт меньше опечаток, чем whisper-small.
* Опробую несколько моделей для исправления ошибок (spell-correction) — как предобученные, так и дообученные на собственном датасете — и выберу лучшую по итоговому WER.
* Возьму эту лучшую модель и улучшу её, адаптировав подход из шага 4: немного изменю данные, стратегию дообучения или гиперпараметры. Обосную, почему выбранная модификация теоретически должна повысить качество, и проверю это эмпирически. По результатам сделаю вывод: удалось ли улучшить WER, а если нет — предположу возможные причины.

In [65]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from evaluate import load

In [61]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [64]:
stt_models = {
    "whisper-small": (
        WhisperProcessor.from_pretrained("openai/whisper-small"),
        WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device),
        "whisper"
    ),
    "wav2vec2-ru": (
        Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-large-ru-golos"),
        Wav2Vec2ForCTC.from_pretrained("bond005/wav2vec2-large-ru-golos").to(device),
        "wav2vec2"
    )
}

In [68]:
test_urls = random.sample(filtered_lines, k=min(10, len(filtered_lines)))

In [71]:
wer_metric = load("wer")

results = {}
for name, (processor, model, model_type) in stt_models.items():
    print(f"\nТестируем модель: {name}")
    predictions = []
    references = []

    for i, url in enumerate(test_urls, 1):
        if url not in ground_truth:
            continue

        ref = ground_truth[url].strip().lower()
        try:
            response = requests.get(url, timeout=30)
            audio_data, sr = sf.read(io.BytesIO(response.content), dtype='float32')

            if audio_data.ndim > 1:
                audio_data = audio_data.mean(axis=1)

            if sr != 16000:
                audio_tensor = torch.tensor(audio_data, dtype=torch.float32)
                audio_resampled = torchaudio.functional.resample(audio_tensor, orig_freq=sr, new_freq=16000)
                audio_data = audio_resampled.numpy()
                sr = 16000

            if model_type == "whisper":
                inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt").input_features.to(device)
                forced_ids = processor.get_decoder_prompt_ids(language="ru", task="transcribe")
                with torch.no_grad():
                    outputs = model.generate(inputs, forced_decoder_ids=forced_ids)
                pred = processor.batch_decode(outputs, skip_special_tokens=True)[0]
            else:  # wav2vec2
                inputs = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True).input_values.to(device)
                with torch.no_grad():
                    logits = model(inputs).logits
                pred_ids = torch.argmax(logits, dim=-1)
                pred = processor.batch_decode(pred_ids)[0]

            pred = pred.strip().lower()
            predictions.append(pred)
            references.append(ref)

            print(f"[{i}] URL: {url}")
            print(f"Истина:     {ground_truth[url]}")
            print(f"Предсказание:     {pred}")

        except Exception as e:
            print(f"[{i}] URL: {url}")
            print(f"Ошибка: {e}")

    # Подсчёт WER
    if predictions:
        wer = wer_metric.compute(predictions=predictions, references=references)
        results[name] = wer
        print(f"WER для {name}: {wer:.4f}")
    else:
        print(f"Нет валидных предсказаний для {name}")


Тестируем модель: whisper-small
[1] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/ca50a690-f1af-4a75-84e7-a8b121eff5e7
Истина:     джон гудман
Предсказание:     джон гудман
[2] URL: http://storage.mds.yandex.net:80/get-voicetoloka/2021744/969c8460-29a8-4c6a-ac71-bc729a6d1f3d
Истина:     криминальное чтиво
Предсказание:     криминальная чтива.
[3] URL: http://storage.mds.yandex.net:80/get-voicetoloka/2021744/6385560c-7068-45c2-9ba7-9e061249e0a4
Истина:     мухаммед али
Предсказание:     мохаммед али
[4] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1879367/af0b23ef-7519-49e0-b32c-cab81f50f2f7
Истина:     крепкий орешек
Предсказание:     крепкий орешек.
[5] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1872575/236b63a8-8f37-444d-8340-25324620e985
Истина:     адвокат дьявола
Предсказание:     адвокат делала
[6] URL: http://storage.mds.yandex.net:80/get-voicetoloka/1879367/4ca1580f-5414-49b8-b816-72f623733105
Истина:     ледниковый период
Предсказание:     